# Modello multimodale 1

Questo modello si occupa di effettuare una classificazione multimodale utilizzando due feature di testo [type, breast] e le immagini per produrre una classificazione binaria: 0 se il tumore è benigno, 1 se il tumore è maligno

## Import delle librerie

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import OneHotEncoder

import albumentations as A

# imposto i seed per la riproducibilità
import random
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

/Volumes/PortableSSD/Università/FDSML/mias/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Parametri e Costanti

In [ ]:
IMG_DIR = 'MIAS-JPEG'
CSV_PATH = 'labels.csv'
PATCH_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 30
LR = 1e-3
NUM_AUG = 5

## Data Augmentation e Preprocessing delle Immagini

In [1]:
alb_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, border_mode=cv2.BORDER_REFLECT_101, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
])

def crop_to_breast(img, padding_ratio=0.05):
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return img
    x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
    pad_w = int(w * padding_ratio)
    pad_h = int(h * padding_ratio)
    x1 = max(x - pad_w, 0)
    y1 = max(y - pad_h, 0)
    x2 = min(x + w + pad_w, img.shape[1])
    y2 = min(y + h + pad_h, img.shape[0])
    return img[y1:y2, x1:x2]

def preprocess_image(path, augment=False):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Immagine non trovata o corrotta: {path}")
    if augment:
        img = alb_transform(image=img)['image']
    img = crop_to_breast(img)
    img = 255 - img
    img = cv2.resize(img, (PATCH_SIZE, PATCH_SIZE))
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img)
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=-1)
    return img


NameError: name 'A' is not defined

## Caricamento e Bilanciamento del Dataset

In [4]:
data = pd.read_csv(CSV_PATH)
data = data[data['severity'].isin(['B', 'M'])]
data['filename'] = data['filename'].astype(str).str.extract(r'(mdb\d{3})')[0].str.strip()
data = data.dropna(subset=['filename'])
data['label'] = data['severity'].map({'B': 0, 'M': 1})

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
meta_features = ohe.fit_transform(data[['type', 'breast']])
data_meta = pd.DataFrame(meta_features, columns=ohe.get_feature_names_out(['type', 'breast']))
data = pd.concat([data.reset_index(drop=True), data_meta.reset_index(drop=True)], axis=1)

train_df, val_df = train_test_split(data, test_size=0.3, stratify=data['label'], random_state=42)

maligni = train_df[train_df.label == 1]
benigni = train_df[train_df.label == 0]
train_df_balanced = pd.concat([benigni, maligni]).reset_index(drop=True)
val_df = val_df.reset_index(drop=True)


## Generatori di dati

In [5]:
def make_generator(df, training=True):
    meta_cols = ohe.get_feature_names_out(['type', 'breast'])
    def generator():
        for idx in range(len(df)):
            row = df.iloc[idx]
            img_path = os.path.join(IMG_DIR, f"{row['filename']}.jpg")
            try:
                label = row['label']
                meta = row[meta_cols].values.astype(np.float32)
                img = preprocess_image(img_path, augment=False)
                yield {"image": img, "meta": meta}, label
                if training:
                    for _ in range(NUM_AUG):
                        img_aug = preprocess_image(img_path, augment=True)
                        yield {"image": img_aug, "meta": meta}, label
            except Exception:
                continue
    return generator


## Creazione Dataset TensorFlow

In [6]:
meta_dim = len(ohe.get_feature_names_out(['type', 'breast']))
train_dataset = tf.data.Dataset.from_generator(
    make_generator(train_df_balanced, training=True),
    output_signature=(
        {
            "image": tf.TensorSpec(shape=(PATCH_SIZE, PATCH_SIZE, 1), dtype=tf.float32),
            "meta": tf.TensorSpec(shape=(meta_dim,), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(), dtype=tf.int64)
    )
).shuffle(256).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE).repeat()

val_dataset = tf.data.Dataset.from_generator(
    make_generator(val_df, training=False),
    output_signature=(
        {
            "image": tf.TensorSpec(shape=(PATCH_SIZE, PATCH_SIZE, 1), dtype=tf.float32),
            "meta": tf.TensorSpec(shape=(meta_dim,), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(), dtype=tf.int64)
    )
).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


## Definizione del Modello CNN

In [7]:
def build_model():
    img_input = keras.Input(shape=(PATCH_SIZE, PATCH_SIZE, 1), name="image")
    meta_input = keras.Input(shape=(meta_dim,), name="meta")
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(img_input)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)
    m = layers.Dense(32, activation='relu')(meta_input)
    m = layers.Dropout(0.2)(m)
    combined = layers.concatenate([x, m])
    combined = layers.Dense(64, activation='relu')(combined)
    combined = layers.Dropout(0.4)(combined)
    outputs = layers.Dense(1, activation='sigmoid')(combined)
    return keras.Model(inputs=[img_input, meta_input], outputs=outputs)

model = build_model()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LR),
    loss=keras.losses.BinaryFocalCrossentropy(gamma=2),
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)


## Training del Modello

In [8]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]

classes = np.unique(train_df_balanced["label"].values)
class_weights_array = compute_class_weight(class_weight='balanced', classes=classes, y=train_df_balanced["label"].values)
class_weights = dict(zip(classes, class_weights_array))

steps_per_epoch = len(train_df_balanced) * (NUM_AUG + 1) // BATCH_SIZE
val_steps = int(np.ceil(len(val_df) / BATCH_SIZE))

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=val_steps,
    callbacks=callbacks,
    class_weight=class_weights
)


Epoch 1/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 202ms/step - accuracy: 0.5311 - loss: 0.2013 - precision: 0.2307 - recall: 0.2385 - val_accuracy: 0.8056 - val_loss: 0.1653 - val_precision: 0.7222 - val_recall: 0.8667 - learning_rate: 0.0010
Epoch 2/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 212ms/step - accuracy: 0.5478 - loss: 0.1955 - precision: 0.2982 - recall: 0.4993 - val_accuracy: 0.4167 - val_loss: 0.2003 - val_precision: 0.4167 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 3/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 187ms/step - accuracy: 0.4846 - loss: 0.2000 - precision: 0.3402 - recall: 0.5501 - val_accuracy: 0.4167 - val_loss: 0.2528 - val_precision: 0.4167 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 4/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 218ms/step - accuracy: 0.5442 - loss: 0.1825 - precision: 0.4029 - recall: 0.6270 - val_accuracy: 0.4167 - val_loss: 0.3370 - val_precision: 0.4167 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 5/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 215ms/step - accuracy: 

## Valutazione del Modello

In [18]:
y_pred_proba = model.predict(val_dataset).flatten()
y_pred = (y_pred_proba > 0.5).astype(int)
y_true = np.concatenate([y.numpy() for (_, _), y in val_dataset])
print(classification_report(y_true, y_pred, target_names=['Benigno', 'Maligno']))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
              precision    recall  f1-score   support

     Benigno       0.89      0.76      0.82        21
     Maligno       0.72      0.87      0.79        15

    accuracy                           0.81        36
   macro avg       0.81      0.81      0.80        36
weighted avg       0.82      0.81      0.81        36



/Volumes/PortableSSD/Università/FDSML/mias/.venv/lib/python3.9/site-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
